# EKE

**GOAL:**  
The code evaluates and ranks multiple model configurations based on their **SST** and **SSS RMSE within eddy-active regions**, where eddies are defined by an **EKE threshold**. It applies an eddy mask, computes RMSEs against observations, and prints rankings to identify the best-performing configurations in those regions.


---

## Imports

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

## Data

In [ ]:
model = xr.open_dataset("../../data/processed/model.nc")
model

In [ ]:
obs = xr.open_dataset("../../data/processed/combined_observations.nc")
obs

In [ ]:
eke = xr.open_dataset("../../data/obs/dt_global_twosat_phy_l4_202307_vDT2021-M01.nc")
eke

In [ ]:
# Define target regular grid (1° resolution)
target_lat = np.linspace(-89.5, 89.5, 180)   # 180 latitudes from -89.5° to 89.5°
target_lon = np.linspace(0.5, 359.5, 360)    # 360 longitudes from 0.5° to 359.5°

# Interpolate EKE data to the target grid using nearest-neighbor method
eke = eke.sel(
    latitude=target_lat,
    longitude=target_lon,
    method="nearest"
).assign_coords(
    latitude=target_lat,
    longitude=target_lon
).rename(
    {'latitude': 'lat', 'longitude': 'lon'}
)["eke"]  # Extract the "eke" variable from the dataset

eke  # Show the interpolated DataArray

## Eddy errors

In [ ]:
# List of configuration identifiers to process
cfgs = ["00", "01", "02", "04", "05", "06", "07", "08", "09", "10", "12", "14", "15", "16", "17"]

# EKE threshold used to define eddy-active regions
eke_thr = 2500

# Print header with threshold information
print("="*60)
print(f"Results for EKE threshold: {eke_thr}")
print("="*60)

# Step 1: Create boolean mask identifying eddy regions (EKE above threshold)
eddy_mask = eke > eke_thr

# Step 2: Plot the eddy mask over a map projection
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(subplot_kw={'projection': proj})
im = eddy_mask.plot(ax=ax, transform=proj, cmap='viridis', add_colorbar=False)
ax.coastlines()
plt.colorbar(im, ax=ax, shrink=0.55)
plt.title(f"Eddy Mask with Coastlines (EKE > {eke_thr})")
plt.show()

# Step 3: Compute RMSE of SST and SSS within eddy regions for each configuration
rmse_results = { "sst": [], "sss": [] }

for cfg in cfgs:
    # Load model output for current configuration
    ds = xr.open_dataset(f"../../data/processed/nemo{cfg}.nc")

    for var in ["sst", "sss"]:
        # Compute squared error and mask to eddy regions
        error = ds[var] - obs[var]
        rmse_eddy = np.sqrt((error.where(eddy_mask)**2).mean())

        # Store RMSE along with configuration label
        rmse_results[var].append((cfg, float(rmse_eddy.values)))

    ds.close()

# Step 4: Print configurations ranked by RMSE (ascending) for each variable
for var in ["sst", "sss"]:
    print("-"*40)
    print(f"Ranking for {var.upper()} (eddy regions, EKE > {eke_thr}):")
    ranked = sorted(rmse_results[var], key=lambda x: x[1])
    for i, (cfg, rmse) in enumerate(ranked, 1):
        print(f"{i}. Config {cfg}: RMSE = {rmse:.3f}")
    print("-"*40)